In [1]:
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import arviz as az
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams["figure.figsize"] = (6,3)

## Hormone data

In [2]:
df = pd.read_csv('horm_sort.csv', usecols = [0,1,2,3])
df[['Testosterone']]= df[['Testosterone']].apply(pd.to_numeric, errors='coerce')
df.rename(columns={"Subjects": "ID"}, inplace = True)
h_df = df.groupby(['Time','Type','ID'])['Testosterone'].min().unstack().T
h_df.columns = ['Pl_T0','T_T0','Pl_T1','T_T1']
h_df.loc['MR24','Pl_T0'] = h_df.loc['MR24','Pl_T1'] 
h_df.loc['MR25','T_T1'] = h_df[h_df['Pl_T0'].between(18, 25)]['T_T1'].mean()
h_df['Pl_change'] = (h_df['Pl_T1']-h_df['Pl_T0'])/h_df['Pl_T0']
h_df['T_change'] = (h_df['T_T1']-h_df['T_T0'])/h_df['T_T0']
h_df['TPl_change'] = (h_df['T_T1']-h_df['Pl_T1'])/h_df['Pl_T1']
TPl_change = h_df.TPl_change
TPl_change['MR23'] = TPl_change.mean()
#TPl_change
ids = pd.read_excel('EE_subj_list.xlsx', header=None, usecols = [1,2])
ids.columns = ['ID', 'Subjects']
ids = ids.join(TPl_change, on = ['ID'])


## Endowment data

In [3]:
df = pd.read_excel('MeMi_Endowment_list.xlsx')
df1 = pd.melt(df, id_vars=['Subjects','Item','Type','Market Price'], value_vars=['WTA_Test','WTA_Plac'], value_name='WTA')
df1['Test'] = 0
df1.loc[df1.variable.str.contains('Test'),'Test'] = 1
df1.drop('variable', axis=1, inplace = True)
df2 = pd.melt(df, id_vars=['Subjects','Item','Type','Market Price'], value_vars=['WTP_Test','WTP_Plac'], value_name='WTP')
df2['Test'] = 0
df2.loc[df2.variable.str.contains('Test'),'Test'] = 1
df2.drop('variable', axis=1, inplace = True)
df = df1.merge(df2)
df.drop(df.index[(df.WTA.isin([0, np.nan])) | (df.WTP.isin([0, np.nan]))], inplace = True)
df['WTA/WTP'] = df.WTA/df.WTP
df = pd.merge(df, ids, on = 'Subjects')
df.loc[df.Test==0, 'TPl_change'] = 0
df = df.drop(['Market Price','WTA','WTP','ID'],axis=1)

Change outliers to max value

In [4]:
max_rel = df['WTA/WTP'].mean()+3*df['WTA/WTP'].std()
df['WTAPcorr'] = df['WTA/WTP']
df.loc[df['WTA/WTP']>max_rel,'WTAPcorr'] = max_rel


In [5]:
df["ItemCodes"] = pd.Categorical(df['Item']).codes
df["SubjectsCode"] = pd.Categorical(df['Subjects']).codes

In [6]:
#df.groupby(['Type','Item','ItemCodes','Test'])['WTA/WTP','WTAPcorr'].mean()

In [7]:
with pm.Model() as model_0:  
    a_item  = pm.Normal('a_item', mu = 1, sd = 2, shape = 27)
    sigma = pm.HalfCauchy('sigma', beta = 1, shape = 27)
    mu = pm.Deterministic('mu', a_item[df.ItemCodes.values])
    
    WTAP = pm.Normal('WTAP', mu=mu, sd=sigma[df.ItemCodes.values], observed=df['WTAPcorr'].values)
    trace_0 = pm.sample(2000, tune=500, init= 'advi')[500:]

IndexError: index 27 is out of bounds for axis 0 with size 27

In [ ]:
az.waic(trace_0)

In [ ]:
with pm.Model() as model_1:  
    a_item  = pm.Normal('a_item', mu = 1, sd = 2, shape = 27)
    sigma = pm.HalfCauchy('sigma', beta = 0.5, shape = 27)

    bT = pm.Normal('bT', mu = 0, sd = 1, shape = 27)
    mu = pm.Deterministic('mu', a_item[df.ItemCodes.values]+bT[df.ItemCodes.values]*df.Test)
    
    WTAP = pm.Normal('WTAP', mu=mu, sd=sigma[df.ItemCodes.values], observed=df['WTAPcorr'].values)
    trace_1 = pm.sample(2000, tune=500, init= 'advi')[500:]

In [ ]:
az.waic(trace_1)

In [ ]:
az.summary(trace_1,var_names=['bT'], credible_interval=.75, round_to=2)

In [ ]:
with pm.Model() as model_2:  
    a_item  = pm.Normal('a_item', mu = 1, sd = 2, shape = 27)
    sigma = pm.HalfCauchy('sigma', beta = 0.5, shape = 27)

    bT = pm.Normal('bT', mu = 0, sd = 1, shape = 27)
    mu = pm.Deterministic('mu', a_item[df.ItemCodes.values]+bT[df.ItemCodes.values]*df.TPl_change)
    
    WTAP = pm.Normal('WTAP', mu=mu, sd=sigma[df.ItemCodes.values], observed=df['WTAPcorr'].values)
    trace_2 = pm.sample(2000, tune=500, init= 'advi')[500:]

In [ ]:
az.waic(trace_2)

In [ ]:
az.summary(trace_2,var_names=['bT'], credible_interval=.75, round_to=2)

In [ ]:
with pm.Model() as model_3:  
    a_item  = pm.Normal('a_item', mu = 1, sd = 2, shape = 27)
    sigma = pm.HalfCauchy('sigma', beta = 0.5, shape = 27)
    sigmaInd = pm.HalfCauchy('sigmaInd', beta = 0.5, shape = 38)

    bT = pm.Normal('bT', mu = 0, sd = 1, shape = 27)
    mu = pm.Deterministic('mu', a_item[df.ItemCodes.values]+bT[df.ItemCodes.values]*df.TPl_change)
    
    WTAP = pm.Normal('WTAP', mu=mu, sd=sigma[df.ItemCodes.values]+sigmaInd[df.SubjectsCode.values], observed=df['WTAPcorr'].values)
    trace_3 = pm.sample(2000, tune=500, init= 'advi')[500:]

In [ ]:
az.summary(trace_3,var_names=['bT'], credible_interval=.75, round_to=2)

In [ ]:
bT = trace_3['bT']
bT_hpd = pm.hpd(bT)

In [ ]:
res_df = pd.DataFrame(df.groupby(['Item'])['ItemCodes'].min()).reset_index()
res_df['bT'] = bT.mean(0)
res_df['bT_low'] = bT_hpd.T[0]
res_df['bT_up'] = bT_hpd.T[1]
type_df = pd.DataFrame(df.groupby('Item')['Type'].min()).reset_index()
res_df = pd.merge(res_df,type_df)

In [ ]:
hed = res_df[res_df.Type == 'h'].reset_index(drop=True)
ut = res_df[res_df.Type == 'u'].reset_index(drop=True)

In [ ]:
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
idx = np.argsort(hed.bT)
y_label = hed.loc[idx,'Item']
y_points = np.linspace(0, 1, len(idx))
x = hed.loc[idx,'bT'].values
xerr = np.abs(np.array([hed.loc[idx,'bT_low'].values,hed.loc[idx,'bT_up'].values])-x)
plt.errorbar(x,y_points,xerr = xerr,fmt='C0o', lw=3, alpha=0.5, mfc = 'purple')
plt.yticks(y_points, y_label);
plt.xlim(-0.8,0.8)
plt.vlines(0, 0, 1, 'grey');
plt.title('Testosterone-related shift in WTA/WTP, hedonic goods', size = 16);
plt.subplot(1,2,2)
idx = np.argsort(ut.bT)
y_label = ut.loc[idx,'Item']
y_points = np.linspace(0, 1, len(idx))
x = ut.loc[idx,'bT'].values
xerr = np.abs(np.array([ut.loc[idx,'bT_low'].values,ut.loc[idx,'bT_up'].values])-x)
plt.errorbar(x,y_points,xerr = xerr,fmt='C0o', lw=3, alpha=0.5, mfc = 'purple')
plt.yticks(y_points, y_label);
plt.xlim(-0.8,0.8)
plt.vlines(0, 0, 1, 'grey');
plt.title('Testosterone-related shift in WTA/WTP, utilitarian goods', size = 16);